In [0]:
%run "/Workspace/Users/ruchika.b.mhetre@v4c.ai/vstone_project/vstone_databricks_pipeline/src/notebooks/00_Setup/project_config"

# Verification of Existing Data

In [0]:
%sql
SELECT COUNT(*) FROM vstone_project.db_project.bronze_transactions;

# Auto Loader Streaming Ingestion (JSON)

In [0]:
from pyspark.sql.functions import lit, current_timestamp

source_path = f"{volume_path}/chunks/chunk3_json"
checkpoint_path = f"{volume_path}/_checkpoints/bronze_json"


In [0]:
# 1. Auto Loader logic with Schema Hints
# This forces the incoming JSON fields to match your table's data types
df_json = (spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "json")
    .option("cloudFiles.schemaLocation", f"{checkpoint_path}_schema")
    .option("cloudFiles.schemaHints", "has_license BOOLEAN, year INT, cost DOUBLE, date DATE")
    .load(source_path)
    .withColumn("source_file", F.lit("chunk3_json"))
    .withColumn("load_timestamp", F.current_timestamp()))

query = (df_json.writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", checkpoint_path)
    .trigger(availableNow=True)
    .toTable(f"{catalog_name}.{schema_name}.bronze_transactions"))

query.awaitTermination() # MANDATORY: Ensures the history record is created
print("JSON Stream committed to history.")